## check the Loc methods and the upload_ ones

In [1]:

import os
import json
import random
import time
import datetime
import logging

import pandas as pd
from typing import Optional

from instagrapi.exceptions import ClientError, ClientJSONDecodeError


from ig_auth import authenticate_and_save_session
from ig_client import IgClient
from ig_data import IgPost, create_post_dataframe
from ig_post_manager import IgPostManager
from config import POSTS_HISTORY_FILE


In [2]:
from instagrapi import Client

In [3]:
from instagrapi.types import Location 

## VPN connect before session start

In [4]:
# 1. Authentication (Always attempt authentication first)
SESSION_FILE = "cl_ig.pkl"
if not os.path.exists(SESSION_FILE):  # Check if session file exists
    authenticate_and_save_session(SESSION_FILE)

# Now, you should have a valid session, so create the client
insta_client = IgClient(SESSION_FILE)
post_manager = IgPostManager(insta_client)

INFO:instagrapi:https://i.instagram.com/api/v1/launcher/sync/
INFO:private_request:johnklanick [200] POST https://i.instagram.com/api/v1/launcher/sync/ (269.0.0.18.75, OnePlus 6T Dev)
INFO:instagrapi:https://i.instagram.com/api/v1/accounts/login/
INFO:private_request:johnklanick [200] POST https://i.instagram.com/api/v1/accounts/login/ (269.0.0.18.75, OnePlus 6T Dev)
INFO:instagrapi:https://i.instagram.com/api/v1/feed/reels_tray/
INFO:private_request:johnklanick [200] POST https://i.instagram.com/api/v1/feed/reels_tray/ (269.0.0.18.75, OnePlus 6T Dev)
INFO:instagrapi:https://i.instagram.com/api/v1/feed/timeline/
INFO:private_request:johnklanick [200] POST https://i.instagram.com/api/v1/feed/timeline/ (269.0.0.18.75, OnePlus 6T Dev)
INFO:ig_auth:Connected Successfully!
INFO:ig_client:Session loaded successfully.


In [5]:
if insta_client.client.user_id:
    print("Logged in as user ID:", insta_client.client.user_id)
else:
    print("Not logged in.")

Logged in as user ID: 67318685241


In [6]:
cl = insta_client.client


In [7]:
if cl.user_id:
    print("Logged in as user ID:", cl.user_id)
else:
    print("Not logged in.")


Logged in as user ID: 67318685241


In [8]:
from instagrapi.exceptions import LoginRequired
try:
    user_info = cl.user_info_by_username('johnklanick') 
    print("Logged in successfully.")
except LoginRequired:
    print("Not logged in or session expired.")


INFO:public_request:[None] [401] GET https://www.instagram.com/johnklanick/?__a=1&__d=dis
INFO:public_request:[None] [401] GET https://www.instagram.com/johnklanick/?__a=1&__d=dis
INFO:public_request:[None] [401] GET https://www.instagram.com/johnklanick/?__a=1&__d=dis
INFO:instagrapi:https://i.instagram.com/api/v1/users/johnklanick/usernameinfo/
INFO:private_request:None [200] GET https://i.instagram.com/api/v1/users/johnklanick/usernameinfo/ (269.0.0.18.75, OnePlus 6T Dev)


Logged in successfully.


###Location

from https://subzeroid.github.io/instagrapi/usage-guide/location.html

In [9]:

def get_location_by_pk(client: Client, location_pk: int, retries: int = 3) -> Optional[Location]:
    """
    Retrieves location information by its primary key (PK) using the Instagrapi API.

    Args:
        client: The Instagrapi Client object for API interactions.
        location_pk: The unique identifier (PK) of the location to retrieve.
        retries: The number of times to retry the request in case of failure (default is 3).

    Returns:
        The Location object if found, or None if not found after the specified retries.
    """

    for attempt in range(retries):
        try:
            location: Location = client.location_info(location_pk)
            return location 
        except ClientJSONDecodeError as e:
            print(f"Attempt {attempt + 1} failed for location PK {location_pk}: {e}")
            if attempt < retries - 1:  # Only sleep if there are more retries left
                delay = 2 ** attempt + random.uniform(0, 1)  # Exponential backoff with jitter
                time.sleep(delay)   # Introduce a delay before retrying

    print(f"Could not retrieve location info for PK {location_pk} after {retries} attempts.")
    return None  # Return None to indicate failure


In [10]:


# Example usage
location = get_location_by_pk(cl, 61047499) 

if location:
    print("Location found:", location)
else:
    print("Location not found.")


INFO:public_request:[None] [201] GET https://www.instagram.com/explore/locations/61047499/?__a=1&__d=dis
ERROR:public_request:Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/explore/locations/61047499/?__a=1&__d=dis) >>> 
INFO:public_request:[None] [201] GET https://www.instagram.com/explore/locations/61047499/?__a=1&__d=dis
ERROR:public_request:Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/explore/locations/61047499/?__a=1&__d=dis) >>> 
INFO:public_request:[None] [201] GET https://www.instagram.com/explore/locations/61047499/?__a=1&__d=dis
ERROR:public_request:Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/explore/locations/61047499/?__a=1&__d=dis) >>> 
INFO:instagrapi:https://i.instagram.com/api/v1/locations/61047499/location_info/
INFO:private_request:None [200] GET https://i.instagram.com/api/v1/locations/61047499/location_info/ (269.0.0.18.75, OnePlus 6T Dev)


Location found: pk=61047499 name='Malecon Cisneros Miraflores' phone='' website='http://www.miraflores.gob.pe' category='Beach' hours={'status': 'Open 24 hours', 'current_status': 'Open 24 hours', 'hours_today': '', 'schedule': [], 'is_open': True} address='' city='Lima, Peru' zip='15074' lng=None lat=None external_id=428571523840202 external_id_source=None


In [11]:
# Assuming 'location' contains the <class 'instagrapi.types.Location'> object
location_dict = {
    "pk": location.pk,
    "name": location.name,
    "phone": location.phone,
    "website": location.website,
    "category": location.category,
    "hours": location.hours,  # Nested dictionary, already in a readable format
    "address": location.address,
    "city": location.city,
    "zip": location.zip,
    "lng": location.lng,
    "lat": location.lat,
    "external_id": location.external_id,
    "external_id_source": location.external_id_source,
}

formatted_json = json.dumps(location_dict, indent=4)
print(formatted_json)


{
    "pk": 61047499,
    "name": "Malecon Cisneros Miraflores",
    "phone": "",
    "website": "http://www.miraflores.gob.pe",
    "category": "Beach",
    "hours": {
        "status": "Open 24 hours",
        "current_status": "Open 24 hours",
        "hours_today": "",
        "schedule": [],
        "is_open": true
    },
    "address": "",
    "city": "Lima, Peru",
    "zip": "15074",
    "lng": null,
    "lat": null,
    "external_id": 428571523840202,
    "external_id_source": null
}


In [12]:
def get_location_by_name(location_name, retries=3):
    for attempt in range(retries):
        try:
            location = cl.fbsearch_places(location_name)  # Or use location_info_a1 if you're sure
            return location
        except ClientJSONDecodeError as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(10)  # Delay before retrying
    raise Exception(f"Could not retrieve location info from name after {retries} attempts.")

location = get_location_by_name('punta umbria')

INFO:instagrapi:https://i.instagram.com/api/v1/fbsearch/places/
INFO:private_request:None [200] GET https://i.instagram.com/api/v1/fbsearch/places/?search_surface=places_search_page&timezone_offset=-14400&lat=40.74&lng=-73.94&count=30&query=punta+umbria (269.0.0.18.75, OnePlus 6T Dev)


In [17]:
# Assuming 'location' contains the <class 'instagrapi.types.Location'> object
location_dict = {
    "pk": location.pk,
    "name": location.name,
    "phone": location.phone,
    "website": location.website,
    "category": location.category,
    "hours": location.hours,  # Nested dictionary, already in a readable format
    "address": location.address,
    "city": location.city,
    "zip": location.zip,
    "lng": location.lng,
    "lat": location.lat,
    "external_id": location.external_id,
    "external_id_source": location.external_id_source,
}

formatted_json = json.dumps(location_dict, indent=4)
print(formatted_json)



{
    "pk": 265297120,
    "name": "El Rocio , Almonte",
    "phone": "",
    "website": "",
    "category": "",
    "hours": {},
    "address": "",
    "city": "",
    "zip": null,
    "lng": -6.47485654398,
    "lat": 37.1771048062,
    "external_id": 210466289080691,
    "external_id_source": "facebook_places"
}


In [14]:
from typing import List
from instagrapi import Client
from instagrapi.exceptions import ClientJSONDecodeError
import time

def get_top_locations_by_name(client: Client, location_name: str, top_n: int = 3, retries: int = 3) -> List:
    """
    Retrieves the top N locations matching the given location name using Instagrapi's fbsearch_places.

    Args:
        client: The Instagrapi Client object for interacting with the API.
        location_name: The name of the location to search for.
        top_n: The number of top results to return (default is 3).
        retries: The number of times to retry the request if it fails (default is 3).

    Returns:
        A list of at most 'top_n' Location objects, or an empty list if none are found or an error occurs.
    """

    for attempt in range(retries):
        try:
            locations: List = client.fbsearch_places(location_name)
            return locations[:top_n]  # Return the top 'top_n' results

        except ClientJSONDecodeError as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < retries - 1:
                time.sleep(10)  # Delay before retrying

    print(f"Could not retrieve location info for '{location_name}' after {retries} attempts.")
    return []  # Return an empty list in case of failure

# ... (Assuming you have your 'cl' client object set up)

top_locations = get_top_locations_by_name(cl, 'El Rocio')
for location in top_locations:
    print(location)

INFO:instagrapi:https://i.instagram.com/api/v1/fbsearch/places/
INFO:private_request:None [200] GET https://i.instagram.com/api/v1/fbsearch/places/?search_surface=places_search_page&timezone_offset=-14400&lat=40.74&lng=-73.94&count=30&query=El+Rocio (269.0.0.18.75, OnePlus 6T Dev)


pk=240482382 name='El Rocío, Andalucia, Spain' phone='' website='' category='' hours={} address='' city='' zip=None lng=-6.48778 lat=37.1333 external_id=110310265653776 external_id_source='facebook_places'
pk=131009990801778 name='El Rocío' phone='' website='' category='' hours={} address='  Calle Ermita del Rocío ' city='El Rocío' zip=None lng=-5.98162 lat=37.4079 external_id=131009990801778 external_id_source='facebook_places'
pk=265297120 name='El Rocio , Almonte' phone='' website='' category='' hours={} address='' city='' zip=None lng=-6.47485654398 lat=37.1771048062 external_id=210466289080691 external_id_source='facebook_places'


In [16]:
if top_locations:
    locations_data = []  # List to store location dictionaries
    for location in top_locations:
        location_dict = {
            "pk": location.pk,
            "name": location.name,
            "phone": location.phone,
            "website": location.website,
            "category": location.category,
            "hours": location.hours,
            "address": location.address,
            "city": location.city,
            "zip": location.zip,
            "lng": location.lng,
            "lat": location.lat,
            "external_id": location.external_id,
            "external_id_source": location.external_id_source,
        }
        locations_data.append(location_dict)

    formatted_json = json.dumps(locations_data, indent=4)
    print(formatted_json)
else:
    print("No locations found.")

[
    {
        "pk": 240482382,
        "name": "El Roc\u00edo, Andalucia, Spain",
        "phone": "",
        "website": "",
        "category": "",
        "hours": {},
        "address": "",
        "city": "",
        "zip": null,
        "lng": -6.48778,
        "lat": 37.1333,
        "external_id": 110310265653776,
        "external_id_source": "facebook_places"
    },
    {
        "pk": 131009990801778,
        "name": "El Roc\u00edo",
        "phone": "",
        "website": "",
        "category": "",
        "hours": {},
        "address": "  Calle Ermita del Roc\u00edo ",
        "city": "El Roc\u00edo",
        "zip": null,
        "lng": -5.98162,
        "lat": 37.4079,
        "external_id": 131009990801778,
        "external_id_source": "facebook_places"
    },
    {
        "pk": 265297120,
        "name": "El Rocio , Almonte",
        "phone": "",
        "website": "",
        "category": "",
        "hours": {},
        "address": "",
        "city": "",
        

In [ ]:
# Assuming 'location' contains the <class 'instagrapi.types.Location'> object
location_dict = {
    "pk": location.pk,
    "name": location.name,
    "phone": location.phone,
    "website": location.website,
    "category": location.category,
    "hours": location.hours,  # Nested dictionary, already in a readable format
    "address": location.address,
    "city": location.city,
    "zip": location.zip,
    "lng": location.lng,
    "lat": location.lat,
    "external_id": location.external_id,
    "external_id_source": location.external_id_source,
}

formatted_json = json.dumps(location_dict, indent=4)
print(formatted_json)


### upload

from https://subzeroid.github.io/instagrapi/usage-guide/media.html
 

### Find Track

at https://subzeroid.github.io/instagrapi/usage-guide/track.html


remember from (https://subzeroid.github.io/instagrapi/ )
Public (anonymous request via web api) methods have a suffix _gql (Instagram GraphQL) or _a1 (example https://www.instagram.com/example/?__a=1)
Private (authorized request via mobile api) methods have _v1 suffix

The first request to fetch media/user is public (anonymous), if instagram raise exception, then use private (authorized).
Example (pseudo-code):

def media_info(media_pk):
    try:
        return self.media_info_gql(media_pk)
    except ClientError as e:
        # Restricted Video: This video is not available in your country.
        # Or media from private account
        return self.media_info_v1(media_pk)